In [1]:
import re
import getpass
import gmaps
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
from IPython.display import clear_output, display
from parse import parse
import pandas as pd

import yaml
import time
import os
import sys

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString
from prognos_tools.Basin import Basin


gmapsToken=b'gAAAAABctxJp5wE73qK6U5VieCi0WXNnNex4KxLZHutsESa8fW9v43lLa1Ag0qsxKFjXXo9MBAdvUpqJPW-QmCE0gH_Opf9g4xAG1VaI2WarO_xDZg44VLMCHkd_6O8ofgp8u4VuFBMr'
key = getpass.getpass('Password: ')
apiKey = decryptString(gmapsToken,key)
gmaps.configure(api_key=apiKey)
del key,apiKey

#Creating folder to store temporary files and downloaded data
saveFolder = './DownloadedData/'

args = [saveFolder,]
!!rm -rf {args[0]} && mkdir -p {args[0]}

Password:  ·······


[]

## Querying extent of the basin
### Creating a VM with the geodatabase on it

In [2]:
machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'prognos-basin'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'geonorway'
machineInfo['imageProject'] = 'nivacatchment'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/threddsBasin'
machineInfo['region'] = 'europe-west2-a'
machineInfo['machineType'] = 'n1-standard-4'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'


#Actually instantiating the virtual machine 
basinGenerator = Basin(machineInfo)
basinGenerator.instantiate(fabfile='fabfile_catchment.py')
print("The ip of the google cloud instance is {}".format(basinGenerator.ip))

#Testing connection to instance
basinGenerator.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**basinGenerator.machineInfo))

Creating instance ljb-basin...
NAME       ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
ljb-basin  europe-west2-a  n1-standard-4               10.154.0.30  35.246.18.90  RUNNING
Created [https://www.googleapis.com/compute/beta/projects/nivacatchment/zones/europe-west2-a/instances/ljb-basin].
 - Disk size: '200 GB' is larger than image size: '20 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/persistent-disks#repartitionrootpd for details.

(b"Generating public/private rsa key pair.\nYour identification has been saved in /home/jose-luis/.ssh/threddsBasin/jose-luis.\nYour public key has been saved in /home/jose-luis/.ssh/threddsBasin/jose-luis.pub.\nThe key fingerprint is:\nSHA256:+Rt7cKOUq5DKsqvVILqRAoU7lOa2zDUd0iTlmt+JdBk jose-luis\nThe key's randomart image is:\n+---[RSA 2048]----+\n|   ..o           |\n| .. =            |\n|.+.. + E 

### Getting basins extent

In [3]:
#Schema were processing will be done
schema = 'metno'

#Putting ljb data as yaml
import pandas as pd
ljb = pd.read_csv('~/Downloads/Stream_Monitoring_Locations_forJLG.csv',encoding='iso-8859-1',header=1)
coords = [{'station': {'station_name': i['Prøvested:'].strip(),
                        'station_id': j,
                        'longitude': i['°E'],
                        'latitude': i['°N'],
                        'epsg': 4326,
                        'buffer': 20000
                       }
          } for j,i in ljb.iterrows()]

for i in coords:
    if i['station']['station_name'] == 'Dypingbekken':
        i['station']['longitude'] = 10.017462
        i['station']['latitude'] = 59.849623
    elif i['station']['station_name'] == 'Sandungsbekken':
        i['station']['longitude'] = 10.048416
        i['station']['latitude'] = 59.854848
    elif i['station']['station_name'] == 'Langevannsbekken':
        i['station']['longitude'] = 10.060717
        i['station']['latitude'] = 59.853656
    elif i['station']['station_name'] == 'Guritjernsbekken':
        i['station']['longitude'] = 10.060792
        i['station']['latitude'] = 59.873171

with open('ljb.yaml','w') as f:
    f.write(yaml.dump(coords,allow_unicode=True))

#Getting basin outlets from yaml file
stationsFile = 'svalbard.yaml'
stations = yaml.safe_load(open(stationsFile))
display(stations)

#Actually processing the basin
geoResultsFolder='./geoResults'
basinGenerator.getBasinLayers(stationsFile,schema,saveFolder=geoResultsFolder)
    
# # Now we delete the basinGenerator vm
basinGenerator.kill()

[{'station': {'buffer': 20000,
   'epsg': 3035,
   'station_id': 1,
   'latitude': 3921738,
   'longitude': 4217991,
   'station_name': 'Birkenes'}}]

Gonna create the getBasin.sh file
./geoResults/getBasin.sh
fab -f fabfile_catchment.py processScript:./geoResults/getBasin.sh,./geoResults/box.txt
[35.246.18.90] Executing task 'processScript'
[35.246.18.90] put: ./geoResults/getBasin.sh -> getBasin.sh
[35.246.18.90] run: chmod +x getBasin.sh
[35.246.18.90] run: ./getBasin.sh
[35.246.18.90] out: NOTICE:  table "stations" does not exist, skipping
[35.246.18.90] out:  initializestations 
[35.246.18.90] out: --------------------
[35.246.18.90] out:  
[35.246.18.90] out: (1 row)
[35.246.18.90] out: 
[35.246.18.90] out:  addstations 
[35.246.18.90] out: -------------
[35.246.18.90] out:  
[35.246.18.90] out: (1 row)
[35.246.18.90] out: 
[35.246.18.90] out: NOTICE:  schema "metno" does not exist, skipping
[35.246.18.90] out:  initializeresultsschema 
[35.246.18.90] out: -------------------------
[35.246.18.90] out:  
[35.246.18.90] out: (1 row)
[35.246.18.90] out: 
[35.246.18.90] out: NOTICE:  table "buffertable" does not exist, skipping
[35

### Displaying basins

In [4]:
with open(os.path.join(geoResultsFolder,'gmaps.txt'), 'r') as f:
    lines = f.read().splitlines()
    
fig = gmaps.figure()
fig.add_layer(gmaps.geojson_layer(yaml.safe_load(lines[0])))

lines[2:] = [i.split(';') for i in lines[2:]]
outlets = [{"name": i[0], "area": i[3]} for i in lines[2:]]
locations = [(float(i[2]),float(i[1])) for i in lines[2:]]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))